In [1]:
!pip install -q langchain langchain-community google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 1.7 MB/s eta 0:00:00


In [2]:
!pip install langchain-google-genai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 kB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 23.3 MB/s eta 0:00:00
  Attempting uninstall: google-ai-generativelanguage
    Found existing installation: google-ai-generativelanguage 0.6.15
    Uninstalling google-ai-generativelanguage-0.6.15:
      Successfully uninstalled google-ai-generativelanguage-0.6.15
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
google-generativeai 0.8.5 requires google-ai-generativelanguage==0.6.15, but you have google-ai-generativelanguage 0.6.18 which is incompatible.


In [3]:
import os
from google.colab import userdata

# Access the API key from Colab's Secrets Manager
os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')

In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.5)

In [10]:
# with structured_output
# typeddict
from typing import TypedDict

class Person(TypedDict):
  name:str
  age:int

new_person: Person = {
    "name":"john doe",
    "age":12
}

print(new_person)

{'name': 'john doe', 'age': 12}


# Structured output for the review - Annotated

In [14]:
from typing import TypedDict

class Review(TypedDict):

  summary: str
  sentiment: str

structured_model =  model.with_structured_output(Review)

result =structured_model.invoke(""" I was a bit skeptical at first, but this product truly surprised me.
From the moment I unboxed it, everything—from the sleek design to the user-friendly
features—felt premium. It performs exactly as advertised and has added so much value to my
 daily routine. I’ve tested similar items in the past, and this one outshines them all. If you’re on the fence,
  don’t hesitate—this is a solid investment. Highly recommended!
""")

print(result)
print(result["sentiment"])
print(result["summary"])



{'sentiment': 'positive', 'summary': 'This product truly surprised me. From the moment I unboxed it, everything—from the sleek design to the user-friendly features—felt premium. It performs exactly as advertised and has added so much value to my daily routine. I’ve tested similar items in the past, and this one outshines them all. If you’re on the fence, don’t hesitate—this is a solid investment. Highly recommended!'}
positive
This product truly surprised me. From the moment I unboxed it, everything—from the sleek design to the user-friendly features—felt premium. It performs exactly as advertised and has added so much value to my daily routine. I’ve tested similar items in the past, and this one outshines them all. If you’re on the fence, don’t hesitate—this is a solid investment. Highly recommended!


# with annotated

In [16]:
from typing import TypedDict, Annotated, Optional ,Literal

class Review(TypedDict):

  key_themes:Annotated[list[str], "write down all the key themes discussed on the review on the list"]
  summary: Annotated[str, "A brief summary of the review"]
  sentiment: Annotated[str, "Return sentiment of the review  either negative , positive or neutral"]
  # sentiment:Annotated[Literal["pos", "neg"], "return the sentiment either positive, negative or neutral"]
  pros:Annotated[Optional[list[str]], "write down all the pros inside the list"]
  cons:Annotated[Optional[list[str]], "write down all the cons inside the list"]



structured_model =  model.with_structured_output(Review)

result =structured_model.invoke(""" The Samsung Galaxy S24 Ultra delivers blazing-fast performance, a jaw-dropping 200MP camera, and excellent battery life with speedy charging.
 Despite its bulk and hefty price, it’s a powerhouse packed with premium features and tools for productivity and creativity, i fucking loved it !!""")

print(result)
print(result.keys())
print(result["sentiment"])
print(result["summary"])

{'pros': ['"blazing-fast performance"', '"jaw-dropping 200MP camera"', '"excellent battery life with speedy charging"', '"powerhouse packed with premium features and tools for productivity and creativity"'], 'sentiment': '"positive"', 'cons': ['"bulk"', '"hefty price"']}
dict_keys(['pros', 'sentiment', 'cons'])
"positive"


KeyError: 'summary'

In [17]:
# error in summary beacuse there is no summary in this review add summary and run again by changing the review....

# Pydantic for data validation

In [18]:
try:
    import pydantic
    print("Pydantic is installed.")
except ImportError:
    print("Pydantic is not installed.")

Pydantic is installed.


In [19]:
# if not: !pip install pydantic

In [54]:
from pydantic import BaseModel, EmailStr, Field
from typing import Optional

class Student(BaseModel):

  # define and set he default value
  name:str = "sanoj"
  age:Optional[int] = None
  email:EmailStr
  cgpa:float = Field(gt=0, lt=10, default=6, description="decimal value representing the CGPA of student")


new_student = {"age":32, "email": "abc@gmail.com", "cgpa":6}

# create a pydantic object
student = Student(**new_student)
print("pydantic object: ",student)


# convert into dictionary
student_dict = dict(student)
print("dict: ",student_dict['age'])

# into json

student_json = student.model_dump_json()
print("json object: ", student_json)

pydantic object:  name='sanoj' age=32 email='abc@gmail.com' cgpa=6.0
dict:  32
json object:  {"name":"sanoj","age":32,"email":"abc@gmail.com","cgpa":6.0}


In [32]:
!pip install pydantic[email]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 313.6/313.6 kB 9.8 MB/s eta 0:00:00


# Pydantic part-2

In [55]:
from typing import TypedDict, Annotated, Optional ,Literal
from pydantic import BaseModel, EmailStr, Field

# pydantic model is inherited form the base model
class Review(BaseModel):

  key_themes: list[str] = Field(description="write down all the key themes discussed on the review on the list")
  summary: str = Field(description="A brief summary of the review")
  sentiment:Literal["pos", "neg"] = Field(description="Return sentiment of the review  either negative , positive or neutral")
  pros:Optional[list[str]] = Field( default=None ,description="write down all the pros inside the list")
  cons:Optional[list[str]] = Field(default=None, description="write down all the cons inside the list")
  name:Optional[str] = Field(default=None, description="write down the name of the reviewer")




structured_model =  model.with_structured_output(Review)

result =structured_model.invoke(""" The MacBook is widely praised for its premium aluminum build, exceptional battery life (up to 24 hours)
, and powerful performance driven by Apple’s M-series chips. Its vibrant Retina display and seamless integration with other Apple devices make it a favorite among students,
 creatives, and professionals. While users appreciate its reliability and sleek design, some drawbacks include the high price point, limited port options (especially
  USB-C only on certain models), and lack of suitability for gaming or hardware upgrades.
 Overall, user sentiment is strongly positive—most view it as a worthwhile investment for productivity, longevity, and creative workflows, despite minor limitations.
 review by Dahal Bhai""")

print(result)
# print(result.keys())
print(result["sentiment"])
print(result["summary"])

key_themes=['"premium aluminum build"', '"exceptional battery life"', '"powerful performance"', '"vibrant Retina display"', '"seamless integration"', '"high price point"', '"limited port options"', '"lack of suitability for gaming"', '"hardware upgrades"'] summary="The MacBook is praised for its premium build, long battery life, powerful performance, and vibrant display. However, it is expensive, has limited ports, and isn't ideal for gaming or upgrades." sentiment='pos' pros=['"premium aluminum build"', '"exceptional battery life"', '"powerful performance"', '"vibrant Retina display"', '"seamless integration"'] cons=['"high price point"', '"limited port options"', '"lack of suitability for gaming"', '"hardware upgrades"'] name='"Dahal Bhai"'


AttributeError: 'Review' object has no attribute 'keys'

# JSON schema

In [58]:
{
    "title": "student",
    "description": "schema about student",
    "type":"object",
    "properties":{
        "name":{"type":"string"},
        "age":{"type":"integer"},

    },
    "required":["name"]

}

{'title': 'student',
 'description': 'schema about student',
 'type': 'object',
 'properties': {'name': {'type': 'string'}, 'age': {'type': 'integer'}},
 'required': ['name']}

In [57]:
print(Student.model_json_schema())

{'properties': {'name': {'default': 'sanoj', 'title': 'Name', 'type': 'string'}, 'age': {'anyOf': [{'type': 'integer'}, {'type': 'null'}], 'default': None, 'title': 'Age'}, 'email': {'format': 'email', 'title': 'Email', 'type': 'string'}, 'cgpa': {'default': 6, 'description': 'decimal value representing the CGPA of student', 'exclusiveMaximum': 10, 'exclusiveMinimum': 0, 'title': 'Cgpa', 'type': 'number'}}, 'required': ['email'], 'title': 'Student', 'type': 'object'}


In [59]:
import json

schema = Student.model_json_schema()

# Convert the dictionary to a JSON string
schema_json_string = json.dumps(schema, indent=2)

# Define the filename
filename = "student_schema.json"

# Write the JSON string to a file
with open(filename, "w") as f:
    f.write(schema_json_string)

print(f"JSON schema saved to {filename}")

JSON schema saved to student_schema.json


In [ ]:
from typing import TypedDict, Annotated, Optional, Literal
from pydantic import BaseModel, Field

from langchain_google_genai import ChatGoogleGenerativeAI
model = ChatGoogleGenerativeAI(model="gemini-1.5-flash-latest", temperature=0.5)


# schema
json_schema = {
  "title": "Review",
  "type": "object",
  "properties": {
    "key_themes": {
      "type": "array",
      "items": {
        "type": "string"
      },
      "description": "Write down all the key themes discussed in the review in a list"
    },
    "summary": {
      "type": "string",
      "description": "A brief summary of the review"
    },
    "sentiment": {
      "type": "string",
      "enum": ["pos", "neg"],
      "description": "Return sentiment of the review either negative, positive or neutral"
    },
    "pros": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the pros inside a list"
    },
    "cons": {
      "type": ["array", "null"],
      "items": {
        "type": "string"
      },
      "description": "Write down all the cons inside a list"
    },
    "name": {
      "type": ["string", "null"],
      "description": "Write the name of the reviewer"
    }
  },
  "required": ["key_themes", "summary", "sentiment"]
}


structured_model = model.with_structured_output(json_schema)

result = structured_model.invoke("""I recently upgraded to the Samsung Galaxy S24 Ultra, and I must say, it’s an absolute powerhouse! The Snapdragon 8 Gen 3 processor makes everything lightning fast—whether I’m gaming, multitasking, or editing photos. The 5000mAh battery easily lasts a full day even with heavy use, and the 45W fast charging is a lifesaver.

The S-Pen integration is a great touch for note-taking and quick sketches, though I don't use it often. What really blew me away is the 200MP camera—the night mode is stunning, capturing crisp, vibrant images even in low light. Zooming up to 100x actually works well for distant objects, but anything beyond 30x loses quality.

However, the weight and size make it a bit uncomfortable for one-handed use. Also, Samsung’s One UI still comes with bloatware—why do I need five different Samsung apps for things Google already provides? The $1,300 price tag is also a hard pill to swallow.

Pros:
Insanely powerful processor (great for gaming and productivity)
Stunning 200MP camera with incredible zoom capabilities
Long battery life with fast charging
S-Pen support is unique and useful

Review by Nitish Singh
""")

print(result)